In [1]:
import requests
from pyquery import PyQuery as pq
import json
from json import JSONDecodeError
from tqdm import tqdm_notebook

In [52]:
url = "https://eda.ru/recepty?page={}"

In [35]:
res = requests.get(url.format(1))

In [47]:
recipes_subcat_list = []
for recipes_cat in pq(res.text).find(".seo-footer .seo-footer__list"):
    recipes_subcats = pq(recipes_cat).find("li.seo-footer__list-item, li.seo-footer__list-title _empty")
    for recipes_subcat in recipes_subcats:
        d = {
            "title": pq(recipes_subcat).text().split("\xa0\xa0")[0],
            "href": pq(recipes_subcat).find("a").attr("href"),
            "num": int(pq(recipes_subcat).text().split("\xa0\xa0")[1])
        }
        recipes_subcat_list.append(d)

In [83]:
data = {}

for rec_cat in recipes_subcat_list:
    print(rec_cat["title"])
    has_items = True
    page = 1
    
    while has_items:
        res = requests.get("https://eda.ru/{cat}?page={page}".format(cat=rec_cat["href"], page=page)).text
        
        if len(pq(res).find(".recipes-page__recipes .tile-list__horizontal-tile")):
            page += 1
            for recipe in pq(res).find(".recipes-page__recipes .tile-list__horizontal-tile"):
                title = pq(recipe).find(".horizontal-tile__item-title").text()
                href = pq(recipe).find(".horizontal-tile__item-title a").attr("href")
                print("Page {}, URL {}".format(page, href), end="\r")
                img_src = pq(recipe).find(".horizontal-tile__preview .lazy-load-container").attr("data-src")
                booked = int(pq(recipe).find(".js-bookmark__counter").text())
                likes = int(pq(recipe).find(".widget-list__like-count").text().split()[0])
                dislikes = int(pq(recipe).find(".widget-list__like-count").text().split()[1])
                time_to_cook = pq(recipe).find(".prep-time").text()
                portions = pq(recipe).find(".js-portions-count-print").text()
                ingredients = []
                for ingredient in pq(recipe).find(".ingredients-list .ingredients-list__content-item"):
                    try:
                        attr = pq(ingredient).attr("data-ingredient-object")
                        ingredients.append(json.loads(attr.replace('""', '"')))
                    except JSONDecodeError as err:
                        print(pq(ingredient).attr("data-ingredient-object"))
                        break
                
                data[href] = {
                    "title": title,
                    "img_src": img_src,
                    "ingredients": ingredients,
                    "booked": booked,
                    "likes": likes,
                    "dislikes": dislikes,
                    "time_to_cook": time_to_cook,
                    "portions": portions,
                    "category": rec_cat["title"]
                }
        else:
            has_items = False

Соленья и консервация
Варенье, URL /recepty/sousy-marinady/marinovannaya-zelenaya-fasol-s-chili-i-limonom-57147346zmarina-29493tom-amaretti-20897
Салаты на зимуecepty/salaty/fruktovyy-salat-s-malinovym-varenem-125580027450613a-54375ryanostyami-80897
Овощные салатыecepty/salaty/zimniy-citrusovyy-salat-s-chernym-percem-syrom-feta-i-myatoy-57132
Салаты с курицейcepty/salaty/salat-s-rukkoloy-svekloy-i-adygeyskim-syrom-125375-tmina-543483738301180-17405era-513630
Теплые салаты/recepty/salaty/kurinyy-salat-s-lapshoy-569875908nij-salat-55524ousom-tahini-57214i-4532432652
Фруктовые салатыcepty/salaty/teplyy-salat-s-veshenkami-avokado-i-molodym-kartofelem-125123koy-s-ukropom-53915a-51363246-537558
Слоеные салатыrecepty/salaty/fruktovyy-salat-s-malinovym-varenem-125580y-zapravkoy-57557enim-50009691361548
Винегрет URL /recepty/salaty/sloenyy-salat-so-svekloy-i-greckim-orehom-68765073hami-4152602135
Салат «Цезарь»recepty/salaty/vinegret-s-pekinskoj-kapustoj-48237-apelsinovim-sokom-53563-27350-2218

Шаурма, URL /recepty/osnovnye-blyuda/ovoshhnoj-kebab-s-garnirom-iz-svezhej-kukuruzi-541028749eksikanskim-tomatnim-sousom-salsa-17473
Мусака, URL /recepty/osnovnye-blyuda/izrailskaya-shaurma-93530m-93526-39993352419
Шницель URL /recepty/osnovnye-blyuda/veganskaja-musaka-s-chesnokom-5408317531hanov-22476
Буррито URL /recepty/osnovnye-blyuda/shnicel-iz-teljatini-54695po-italjanski-55138esnochnim-sousom-31668724
Биточки URL /recepty/zavtraki/burrito-s-kolbasoj-dlja-zavtraka-55506kapustoj-54193
Клецки, URL /recepty/osnovnye-blyuda/veganskie-bitochki-iz-chechevici-36003-luka-koriandra-18745
Манты3, URL /recepty/osnovnye-blyuda/klecki-s-limonom-i-lukovym-sousom-pesto-57373matnim-sousom-55295-37893
БифштексURL /recepty/osnovnye-blyuda/manti-s-kuricej-govjadinoj-i-kartofelem-43235
Бефстроганов/recepty/osnovnye-blyuda/bifshteks-s-perepelinim-jajcom-49868-42272so-specijami-20323m-48867-feta-40432
Цыпленок табакаcepty/osnovnye-blyuda/dieticheskij-befstroganov-s-jogurtom-40084d-slivochno-gorchichni

In [3]:
len(data)

20759

In [99]:
for num, d in enumerate(data):
    data[d]["id"] = num

In [101]:
json.dump(data, open("recipes.json", "wt", encoding="utf8"))

In [2]:
data = json.load(open("recipes.json", "rt", encoding="utf8"))

In [106]:
df = pd.DataFrame.from_dict(data, orient="index")

In [108]:
df["img_src"].isnull().sum()

4656

In [ ]:
df.to_msgpack("recipes.msg")

In [7]:
for url, d in tqdm_notebook(data.items()):
#     print(d, end="\r")
    if d["img_src"]:
        if d["id"] in range(14419, 40000): # для возобновления закачки с какого-либо момента
            url = "https:" + d["img_src"]
            res = requests.get(url)
            path = "imgs/" + str(d["id"]) + "." + d["img_src"].split(".")[-1]
            open(path, 'wb').write(res.content)